# 옮긴이 노트

## 1 클래스 벡터(정수)를 이진 클래스 행렬로 변환
- 예를 들어, `categorical_crossentropy`와 같이 사용하기 위해

**서명:**
```python
keras.utils.to_categorical(y, num_classes=None, dtype='float32')

```

- 인자:
    - `y`: 행렬로 변환될 클래스 값 갖는 배열 (0에서 `num_classes - 1`까지의 정수)
    - `num_classes`: 클래스의 총 개수. `None`이면, 이것은 `max(y) + 1`로 추론.
    - `dtype`: 입력에서 예상하는 데이터 유형. 기본값: `'float32'`.

- 반환
    - 입력의 이진 행렬 표현. 클래스 축은 마지막에 위치.

**소스:**

```python
@keras_export("keras.utils.to_categorical")
def to_categorical(y, num_classes=None, dtype="float32"):
    y = np.array(y, dtype="int")
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical
```

**예제:**

In [ ]:
from tensorflow.keras.utils import to_categorical

a = to_categorical([0, 1, 2, 3], num_classes=4)
a = tf.constant(a, shape=[4, 4])
print(a)

In [ ]:
from tensorflow.keras.backend import categorical_crossentropy
import numpy as np

b = tf.constant([.9, .04, .03, .03,
                 .3, .45, .15, .13,
                 .04, .01, .94, .05,
                 .12, .21, .5, .17],
                 shape=[4, 4])
loss = categorical_crossentropy(a, b)
print(np.around(loss, 5))

In [ ]:
np.sum(loss)

In [ ]:
loss = categorical_crossentropy(a, a)
print(np.around(loss, 5))

## 2 로짓을 가진 시그모이드 교차 엔트로피
- 주어진 `logits`으로 시그모이드 교차 엔트로피 계산

**서명:**
    
```python
tf.nn.sigmoid_cross_entropy_with_logits(
    labels=None,
    logits=None,
    name=None,
)
```

- 인자:
    - `labels`: `logits`과 동일한 유형 및 형태. 배타적으로 0에서 1 사이.
    - `logits`: `float32` 또는 `float64` 유형의 텐서. 모든 실수.
    - `name`: 연산에 대한 이름 (선택사양).

- 반환:
    - 구성 요소별 로지스틱 손실이 있는 'logits'과 동일한 형태의 'Tensor'.

- 발생:
    - `ValueError`: `logits` 및 `labels`이 동일한 형태를 가지고 있지 않으면.

**설명:**

각 결과가 독립적이고 완전하게 특정 레이블을 가질 필요가 없는 두 결과가 있는 작업에서 확률 오차를 측정한다.
예를 들어, 사건 발생 확률을 알고 레이블로 사용하는 회귀를 수행할 수 있다. 이 손실은 이진 분류에도 사용될 수 있다.
이 손실은 레이블이 0 또는 1인 이진 분류에도 사용될 수 있다.

  

간략하게, `x = logits`, `z = labels` 하자. 로지스틱 손실은

        z * -log(sigmoid(x)) + (1 - z) * -log(1 - sigmoid(x))
      = z * -log(1 / (1 + exp(-x))) + (1 - z) * -log(exp(-x) / (1 + exp(-x)))
      = z * log(1 + exp(-x)) + (1 - z) * (-log(exp(-x)) + log(1 + exp(-x)))
      = z * log(1 + exp(-x)) + (1 - z) * (x + log(1 + exp(-x))
      = (1 - z) * x + log(1 + exp(-x))
      = x - x * z + log(1 + exp(-x))

x < 0 대하여, exp(-x)에서 과다를 방지하기 위해 위의 공식을 재구성한다:

        x - x * z + log(1 + exp(-x))
      = log(exp(x)) - x * z + log(1 + exp(-x))
      = - x * z + log(1 + exp(x))

따라서, 안정성을 보장하고 오버플로를 방지하기 위해, 구현은 이와 동등한 공식을 사용한다.

      max(x, 0) - x * z + log(1 + exp(-abs(x)))

`logits` 및 `labels` 반드시 동일 유형과 형태를 가져야 한다.

In [ ]:
logits = tf.constant([1., -1., 0., 1., -1., 0., 0.])
labels = tf.constant([0., 0., 0., 1., 1., 1., 0.5])
tf.nn.sigmoid_cross_entropy_with_logits(labels=labels, logits=logits).numpy()

여러 결과를 처리하는 손실과 비교하여, 일반적인 다중 클래스 분류를 위한 `tf.nn.softmax_cross_entropy_with_logits` 및 하드 레이블이 있는 보다 효율적인 다중 클래스 분류를 위한 `tf.nn.sparse_softmax_cross_entropy_with_logits`는 이진 분류를 약간 단순화한 것이다:

        sigmoid(x) = softmax([x, 0])[0]

  $$\frac{1}{1 + e^{-x}} = \frac{e^x}{e^x + e^0}$$

`sigmoid_cross_entropy_with_logits`는 (0과 1 사이의 확률) 소프트 이진 레이블에 대해 작동하지만, 레이블이 어려운 이진 분류에도 사용할 수 있다.
모든 세 기호 사이에 동일한 경우에, 확률 0은 두 번째 클래스를 가리키거나 또는 1은 첫 번째 클래스를 가리킨다:

In [ ]:
sigmoid_logits = tf.constant([1., -1., 0.])
softmax_logits = tf.stack([sigmoid_logits, tf.zeros_like(sigmoid_logits)], axis=-1)
soft_binary_labels = tf.constant([1., 1., 0.])
soft_multiclass_labels = tf.stack([soft_binary_labels, 1. - soft_binary_labels], axis=-1)
hard_labels = tf.constant([0, 0, 1])
tf.nn.sparse_softmax_cross_entropy_with_logits(labels=hard_labels, logits=softmax_logits).numpy()

In [ ]:
tf.nn.softmax_cross_entropy_with_logits(labels=soft_multiclass_labels, logits=softmax_logits).numpy()

In [ ]:
tf.nn.sigmoid_cross_entropy_with_logits(labels=soft_binary_labels, logits=sigmoid_logits).numpy()

**소스:**
    
```python
@tf_export("nn.sigmoid_cross_entropy_with_logits", v1=[])
@dispatch.register_binary_elementwise_api
@dispatch.add_dispatch_support
def sigmoid_cross_entropy_with_logits_v2(  # pylint: disable=invalid-name
    labels=None,
    logits=None,
    name=None):
  return sigmoid_cross_entropy_with_logits(
      logits=logits, labels=labels, name=name)


@tf_export(v1=["nn.sigmoid_cross_entropy_with_logits"])
@dispatch.add_dispatch_support
def sigmoid_cross_entropy_with_logits(  # pylint: disable=invalid-name
    _sentinel=None,
    labels=None,
    logits=None,
    name=None):
  """sigmoid_cross_entropy_with_logits_v2 참조."""
  # pylint: disable=protected-access
  nn_ops._ensure_xent_args("sigmoid_cross_entropy_with_logits", _sentinel,
                           labels, logits)
  # pylint: enable=protected-access

  with ops.name_scope(name, "logistic_loss", [logits, labels]) as name:
    logits = ops.convert_to_tensor(logits, name="logits")
    labels = ops.convert_to_tensor(labels, name="labels")
    try:
      labels.get_shape().assert_is_compatible_with(logits.get_shape())
    except ValueError:
      raise ValueError("`logits` and `labels` must have the same shape, "
                       f"received ({logits.get_shape()} vs "
                       f"{labels.get_shape()}).")

    # 위의 로지스틱 손실 공식
    #   x - x * z + log(1 + exp(-x))
    # x < 0 대하여, 좀 더 수치적으로 안정된 공식은
    #   -x * z + log(1 + exp(x))
    # 이들 두 표현식은 다음과 같이 결합될 수 있다:
    #   max(x, 0) - x * z + log(1 + exp(-abs(x)))
    # 0에서 경사를 계산할 수 있도록 max 및 abs 함수의 사용자 지정 버전을 정의.
    zeros = array_ops.zeros_like(logits, dtype=logits.dtype)
    cond = (logits >= zeros)
    relu_logits = array_ops.where(cond, logits, zeros)
    neg_abs_logits = array_ops.where(cond, -logits, logits)  # pylint: disable=invalid-unary-operand-type
    return math_ops.add(
        relu_logits - logits * labels,
        math_ops.log1p(math_ops.exp(neg_abs_logits)),
        name=name)
```

## 3 범주형 교차 엔트로피 (backend)
- 출력 텐서와 목표 텐서 사이의 범주형 교차 엔트로피

**서명:**

```python
keras.backend.categorical_crossentropy(
    target,
    output,
    from_logits=False,
    axis=-1,
)
```

- 인자:
    - `target`: `output`와 동일 형태의 텐서
    - `output`: 소프트맥스의 텐서 결과 (`from_logits`가 참이 아니면, 이 경우 `output`은 로짓이 될 것으로 예상)
    - `from_logits`: 참/거짓, `output`이 소프트맥스의 결과 또는 로짓의 텐서인지 여부
    - `axis`: 채널 축을 지정하는 정수. `axis=-1`은 데이터 포맷 `channels_last`이고, `axis=1`은 `channels_first`에 상응
- 반환:
    - 출력 텐서
- 발생:
    - `ValueError`: `axis`가 -1도 아니고 `output`의 축 중 하나도 아닌 경우.

**소스:**

```python
@keras_export("keras.backend.categorical_crossentropy")
@tf.__internal__.dispatch.add_dispatch_support
def categorical_crossentropy(target, output, from_logits=False, axis=-1):
    target = tf.convert_to_tensor(target)
    output = tf.convert_to_tensor(output)
    target.shape.assert_is_compatible_with(output.shape)

    output, from_logits = _get_logits(
        output, from_logits, "Softmax", "categorical_crossentropy"
    )
    if from_logits:
        return tf.nn.softmax_cross_entropy_with_logits(
            labels=target, logits=output, axis=axis
        )

    # 각 샘플의 확률 클래스 합계가 1이 되도록 예측를 조정
    output = output / tf.reduce_sum(output, axis, True)
    # 확률로 부터 교차 엔트로피 계산
    epsilon_ = _constant_to_tensor(epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, epsilon_, 1.0 - epsilon_)
    return -tf.reduce_sum(target * tf.math.log(output), axis)
```

**예제:**

In [ ]:
import tensorflow as tf

a = tf.constant([1., 0., 0., 0., 1., 0., 0., 0., 1.], shape=[3,3])
print(a)

In [ ]:
b = tf.constant([.9, .05, .05, .05, .89, .06, .05, .01, .94], shape=[3, 3])
print(b)

In [ ]:
from tensorflow.keras.backend import categorical_crossentropy

loss = categorical_crossentropy(a, b)
print(np.around(loss, 5))

In [ ]:
loss = categorical_crossentropy(a, a)
print(np.around(loss, 5))

In [ ]:
keras.metrics.categorical_crossentropy??

## 4 범주형 교차 엔트로피 (metrics & losses)
- 범주형 교차 엔트로피 손실 계산

### 4.1 `tf.keras.[metrics|losess].categorical_crossentropy`

**서명:**

```python
categorical_crossentropy(
    y_true,
    y_pred,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
)
```

- 인자:
  - y_true: 원-핫 참 목표의 텐서.
  - y_pred: 예측된 목표의 텐서.
  - from_logits: `y_pred`가 로짓 텐서인지 여부. 기본적으로 `y_pred`가 확률 분포를 인코딍한다고 가정.
  - label_smoothing: [0, 1] 범위의 부동소수점. `0`보다 크면 레이블을 부드럽게 한다. 
    예를 들어, `0.1`인 경우 비대상 레이블에 `0.1 / num_classes`를 사용하고 대상 레이블에 `0.9 + 0.1 / num_classes`를 사용.
  - axis: 기본값은 -1. 엔트로피가 계산되는 차원.

- 반환:
  - 범주형 교차 엔트로피 값.

**소스:**

```python
@keras_export(
    "keras.metrics.categorical_crossentropy",
    "keras.losses.categorical_crossentropy",
)
@tf.__internal__.dispatch.add_dispatch_support
def categorical_crossentropy(
    y_true, y_pred, from_logits=False, label_smoothing=0.0, axis=-1
):
    if isinstance(axis, bool):
        raise ValueError(
            f"`axis` must be of type `int`. "
            f"Received: axis={axis} of type {type(axis)}"
        )
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    label_smoothing = tf.convert_to_tensor(label_smoothing, dtype=y_pred.dtype)

    def _smooth_labels():
        num_classes = tf.cast(tf.shape(y_true)[-1], y_pred.dtype)
        return y_true * (1.0 - label_smoothing) + (
            label_smoothing / num_classes
        )

    y_true = tf.__internal__.smart_cond.smart_cond(
        label_smoothing, _smooth_labels, lambda: y_true
    )

    return backend.categorical_crossentropy(
        y_true, y_pred, from_logits=from_logits, axis=axis
    )
```

**표준 사용법:**

In [ ]:
import tensorflow as tf

y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
loss.numpy()

In [ ]:
tf.keras.losses.CategoricalCrossentropy??

### 4.2 `tf.keras.metrics.CategoricalCrossentropy`
- 레이블과 예측 사이의 교차 엔트로피 측정항목을 계산
- 이것은 (2 이상) 다중 레이블에 사용되는 교차 엔트로피 측정항목 클래스이다. 여기세서 레이블은 `원-핫` 표현으로 주어진다고 가정한다. 가령, 레이블 값이 [2, 0, 1] 이면, `y_true` = [[0, 0, 1], [1, 0, 0], [0, 1, 0]].

**`compile()` API에서 사용:**

```python
model.compile(
  optimizer='sgd',
  loss='mse',
  metrics=[tf.keras.metrics.CategoricalCrossentropy()])
```

**서명:**

```python
tf.keras.metrics.CategoricalCrossentropy(
    name='categorical_crossentropy',
    dtype=None,
    from_logits=False,
    label_smoothing=0,
    axis=-1,
)
```

- 인자:
  - `name`: (선택사양) 측정항목 인스턴스의 문자열 이름.
  - `dtype`: (선택사양) 측정항목 결과의 데이터 유형.
  - `from_logits`: (선택사양) 출력의 로직 텐서 여부. 기본적으로, 출력은 확률 분포를 인코딩한다고 가정한다.
  - `label_smoothing`: (선택사양) [0, 1] 범위의 부동소수점.  
     0 보다 크면, 레이블 값은 스므딩되고, 레이블 값에 대한 신뢰도가 완화된다. 
     가령, `label_smoothing=0.2`는 레이블 `0`에 `0.1` 값을 사용하고 `1` 레이블에 `0.9` 값을 사용함을 의미한다.
  - `axis`: (선택사양) 기본값은 -1. 엔트로피가 계산되는 차원.

**소스:**

```python
@keras_export("keras.metrics.CategoricalCrossentropy")
class CategoricalCrossentropy(base_metric.MeanMetricWrapper):
    @dtensor_utils.inject_mesh
    def __init__(
        self,
        name="categorical_crossentropy",
        dtype=None,
        from_logits=False,
        label_smoothing=0,
        axis=-1,
    ):
        super().__init__(
            categorical_crossentropy,
            name,
            dtype=dtype,
            from_logits=from_logits,
            label_smoothing=label_smoothing,
            axis=axis,
        )
```

**독립 실행형 사용:**

In [ ]:
# EPSILON = 1e-7, y = y_true, y` = y_pred
# y` = clip_ops.clip_by_value(output, EPSILON, 1. - EPSILON)
# y` = [[0.05, 0.95, EPSILON], [0.1, 0.8, 0.1]]
# xent = -sum(y * log(y'), axis = -1)
#      = -((log 0.95), (log 0.1))
#      = [0.051, 2.302]
# Reduced xent = (0.051 + 2.302) / 2
m = tf.keras.metrics.CategoricalCrossentropy()
m.update_state([[0, 1, 0], [0, 0, 1]], 
               [[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
m.result().numpy()

In [ ]:
m.reset_state()
m.update_state([[0, 1, 0], [0, 0, 1]],
               [[0.05, 0.95, 0], [0.1, 0.8, 0.1]],
               sample_weight=tf.constant([0.3, 0.7]))
m.result().numpy()

### 4.3 `tf.keras.losses.CategoricalCrossentropy`
- 레이블과 예측 사이의 교차 엔트로피 측정항목을 계산
- 이것은 두 개이상의 다중 레이블에 사용되는 교차 엔트로피 측정항목 클래스이다. 여기세서 레이블은 `원-핫` 표현으로 주어진다고 가정한다. 정수로 레이블이 제공된다면, `SparseCategoricalCrossentropy` 손실을 사용하라. 특성당 `# 클래스` 부동소수점 있어야 한다.
- 아래 단편적인 내용에서, 예제당 `# 클래스` 부동 소수점 값이 있다.

**`compile()` API에서 사용:**

```python
model.compile(optimizer='sgd',
              loss=tf.keras.losses.CategoricalCrossentropy())
```

**서명:**

```python
tf.keras.losses.CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction='auto',
    name='categorical_crossentropy',
)
```

- 인자:
  - `from_logits`: `y_pred`의 로짓 텐서 여부. 
    기본적으로, `y_pred`가 확률 분포를 인코딩한다고 가정.
  - `label_smoothing`: [0, 1] 범위의 부동소수점. 
    0 보다 크면, 레이블 값은 스므딩되고, 레이블 값의 신뢰도는 완화됨을 의미한다.
    예를 들면, `0.1`이면, 목표 레이블에 대하여 `0.1 / num_classes`에 목표 레이블에 대하여 `0.9 + 0.1 / num_classes`을 사용하라.
  - `axis`: 교차 엔트로피를 계산하기 위한 축 (특성 축). 기본값은 -1.
  - `reduction`: 손실에 적용할 `tf.keras.losses.Reduction`의 유형. 기본값은 `AUTO`.
    `AUTO` indicates that the reduction option will be determined by the usage context. 
    대부분이 경우 이것의 기본값은 `SUM_OVER_BATCH_SIZE`이다.
    `tf.distribute.Strategy`이 사용되면, `tf.keras` `compile` 및 `fit`와 같은 내장 훈련 루프 밖에서 `AUTO` 및 `SUM_OVER_BATCH_SIZE` 사용하면 오류가 발생될 것이다. 자세한 사항은 사용자 훈련 [자습서](https://www.tensorflow.org/tutorials/distribute/custom_training) 참조하라.
  - `name`: 인스턴스의 선택사양 이름. 기본값은 'categorical_crossentropy'.


**소스:**

```python
@keras_export("keras.losses.CategoricalCrossentropy")
class CategoricalCrossentropy(LossFunctionWrapper):
    def __init__(
        self,
        from_logits=False,
        label_smoothing=0.0,
        axis=-1,
        reduction=losses_utils.ReductionV2.AUTO,
        name="categorical_crossentropy",
    ):
        super().__init__(
            categorical_crossentropy,
            name=name,
            reduction=reduction,
            from_logits=from_logits,
            label_smoothing=label_smoothing,
            axis=axis,
        )
```

**독립 실행형 사용:**

In [ ]:
y_true = [[0, 1, 0], [0, 0, 1]]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# Using 'auto'/'sum_over_batch_size' reduction type.
cce = tf.keras.losses.CategoricalCrossentropy()
cce(y_true, y_pred).numpy()

In [ ]:
# Calling with 'sample_weight'.
cce(y_true, y_pred, sample_weight=tf.constant([0.3, 0.7])).numpy()

In [ ]:
# Using 'sum' reduction type.
cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.SUM)
cce(y_true, y_pred).numpy()

In [ ]:
# Using 'none' reduction type.
cce = tf.keras.losses.CategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
cce(y_true, y_pred).numpy()

## 5 희소 범주형 교차 엔트로피
- 희소 범주형 교차 엔트로피 손실을 계산

### 5.1 tf.keras.backend.sparse_categorical_crossentropy
- 정수 목표를 갖는 범주형 교차 엔트로피

**서명:**

```python
tf.keras.backend.sparse_categorical_crossentropy(
    target,
    output,
    from_logits=False,
    axis=-1,
    ignore_class=None,
)
```

- 인자:
    - `target`: 정수 텐서.
    - `output`: 소프트맥스의 텐서 결과 (`from_logits`가 참이 아니면, 이 경우 `output`은 로짓이 될 것으로 예상)
    - `from_logits`: 참/거짓, `output`이 소프트맥스의 결과 또는 로짓의 텐서인지 여부
    - `axis`: 채널 축을 지정하는 정수. `axis=-1`은 데이터 포맷 `channels_last`이고, `axis=1`은 `channels_first`에 상응
    - `ignore_class`: 선택적으로 정수. 손실 계산중에 ID는 무시된다.
       이것은 유용한데, 예를 들어, 분할 맵에서 "void" 클래스(일반적으로 -1 또는 255)를 특성으로 하는 분할 문제에 유용하다.
       기본적으로 (`ignore_class=None`), 모든 클래스는 고려된다.

- Returns:
    - 츨략 텐서.

- Raises:
    - `ValueError`: if `axis`가 -1 아니거나 `output`의 축 중 하나가 아닐 경우.

**소스:**

```python
@keras_export("keras.backend.sparse_categorical_crossentropy")
@tf.__internal__.dispatch.add_dispatch_support
@doc_controls.do_not_generate_docs
def sparse_categorical_crossentropy(
    target, output, from_logits=False, axis=-1, ignore_class=None
):
    target = tf.convert_to_tensor(target)
    output = tf.convert_to_tensor(output)

    target = cast(target, "int64")

    output, from_logits = _get_logits(
        output, from_logits, "Softmax", "sparse_categorical_crossentropy"
    )
    if not from_logits:
        epsilon_ = _constant_to_tensor(epsilon(), output.dtype.base_dtype)
        output = tf.clip_by_value(output, epsilon_, 1 - epsilon_)
        output = tf.math.log(output)

    # Permute output so that the last axis contains the logits/probabilities.
    if isinstance(output.shape, (tuple, list)):
        output_rank = len(output.shape)
    else:
        output_rank = output.shape.ndims
    if output_rank is not None:
        axis %= output_rank
        if axis != output_rank - 1:
            permutation = list(
                itertools.chain(
                    range(axis), range(axis + 1, output_rank), [axis]
                )
            )
            output = tf.compat.v1.transpose(output, perm=permutation)
    elif axis != -1:
        raise ValueError(
            "Cannot compute sparse categorical crossentropy with `axis={}` "
            "on an output tensor with unknown rank".format(axis)
        )

    # Try to adjust the shape so that rank of labels = rank of logits - 1.
    output_shape = tf.shape(output)
    target_rank = target.shape.ndims

    update_shape = (
        target_rank is not None
        and output_rank is not None
        and target_rank != output_rank - 1
    )
    if update_shape:
        target = flatten(target)
        output = tf.reshape(output, [-1, output_shape[-1]])

    if ignore_class is not None:
        valid_mask = tf.not_equal(target, cast(ignore_class, target.dtype))
        target = target[valid_mask]
        output = output[valid_mask]

    if py_any(_is_symbolic_tensor(v) for v in [target, output]):
        with get_graph().as_default():
            res = tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels=target, logits=output
            )
    else:
        res = tf.nn.sparse_softmax_cross_entropy_with_logits(
            labels=target, logits=output
        )

    if ignore_class is not None:
        res_shape = cast(output_shape[:-1], "int64")
        valid_mask = tf.reshape(valid_mask, res_shape)
        res = tf.scatter_nd(tf.where(valid_mask), res, res_shape)
        res._keras_mask = valid_mask

        return res

    if update_shape and output_rank >= 3:
        # If our output includes timesteps or
        # spatial dimensions we need to reshape
        res = tf.reshape(res, output_shape[:-1])

    return res

```

### 5.2 `tf.keras.[metrics|losses].sparse_categorical_crossentropy`

**서명:**

```python
tf.keras.metrics.sparse_categorical_crossentropy(
    y_true,
    y_pred,
    from_logits=False,
    axis=-1,
    ignore_class=None,
)
```

- 인자:
  - `y_true`: 실측값.
  - `y_pred`: 예측값.
  - `from_logits`: `y_pred`의 로짓 텐서 여부. 기본적으로, `y_pred`가 확률 분포를 인코딩한다고 가정.
  - `axis`: 기본값은 -1. 엔트로피가 계산되는 축.
  - `ignore_class`: 선택적으로 정수. 손실 계산중에 ID는 무시된다.
     이것은 유용한데, 예를 들어, 분할 맵에서 "void" 클래스(일반적으로 -1 또는 255)를 특성으로 하는 분할 문제에 유용하다.
     기본적으로 (`ignore_class=None`), 모든 클래스는 고려된다.

- 반환:
  - 희소 범주형 교차 엔트로피 손실 값.



**소스:**

```python
@keras_export(
    "keras.metrics.sparse_categorical_crossentropy",
    "keras.losses.sparse_categorical_crossentropy",
)
@tf.__internal__.dispatch.add_dispatch_support
def sparse_categorical_crossentropy(
    y_true, y_pred, from_logits=False, axis=-1, ignore_class=None
):
    return backend.sparse_categorical_crossentropy(
        y_true,
        y_pred,
        from_logits=from_logits,
        ignore_class=ignore_class,
        axis=axis,
    )
```

**독립 실행형 사용:**

In [ ]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
loss.numpy()

In [ ]:
y_true = [[[ 0,  2],
           [-1, -1]],
          [[ 0,  2],
           [-1, -1]]]
y_pred = [[[[1.0, 0.0, 0.0], [0.0, 0.0, 1.0]],
           [[0.2, 0.5, 0.3], [0.0, 1.0, 0.0]]],
          [[[1.0, 0.0, 0.0], [0.0, 0.5, 0.5]],
           [[0.2, 0.5, 0.3], [0.0, 1.0, 0.0]]]]
loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, ignore_class=-1)
loss.numpy()

### 5.3 `tf.keras.metrics.SparseCategoricalCrossentropy`

- 두 개 이상의 레이블 클래스가 있는 경우 이 교차 엔트로피 측정항목을 사용하라.
  레이블은 정수로 제공될 것으로 예상된다. `원-핫` 표현을 사용하여 레이블을 제공하려면 `CategoricalCrossentropy` 측정항목을 사용하라.
  `y_pred`에 대해 특성당 `# 클래스` 부동소수점 값이 있어야 하고 `y_true`에 대해 특성당 단일 부동소수점 값이 있어야 한다.

- 아래 단편적인 내용에서, `y_true`에 대한 예제당 단일 부동 소수점 값과 `y_pred`에 대한 예제당 `# 클래스` 부동 소수점 값이 있다.
  `y_true`의 형태는 `[batch_size]` 이고 `y_pred`의 형태는 `[batch_size, num_classes]`.


- `compile()` API에서:

```python
model.compile(
  optimizer='sgd',
  loss='mse',
  metrics=[tf.keras.metrics.SparseCategoricalCrossentropy()])
```

**서명:**

```python
tf.keras.metrics.SparseCategoricalCrossentropy(
    name: str = 'sparse_categorical_crossentropy',
    dtype: Union[str, tensorflow.python.framework.dtypes.DType, NoneType] = None,
    from_logits: bool = False,
    ignore_class: Optional[int] = None,
    axis: int = -1,
)
```

- Args:
  - name: (선택적) 측정항목 인스턴스 문자열 이름.
  - dtype: (선택적) 측정항목 결과의 데이터 유형.
  - from_logits: (선택적)y_pred`가 로짓 텐서로 예상되는지 여부. 기본적으로, `y_pred`가 확률 분포를 인코드한다고 가정.
  - ignore_class: 선택적으로 정수. 손실 계산중에 ID는 무시된다.
     이것은 유용한데, 예를 들어, 분할 맵에서 "void" 클래스(일반적으로 -1 또는 255)를 특성으로 하는 분할 문제에 유용하다.
     기본적으로 (`ignore_class=None`), 모든 클래스는 고려된다.
  - axis: (선택적) 기본값은 -1. 엔트로피가 계산되는 차원.

**소스:**

```python
@keras_export("keras.metrics.SparseCategoricalCrossentropy")
class SparseCategoricalCrossentropy(base_metric.MeanMetricWrapper):
    @dtensor_utils.inject_mesh
    def __init__(
        self,
        name: str = "sparse_categorical_crossentropy",
        dtype: Optional[Union[str, tf.dtypes.DType]] = None,
        from_logits: bool = False,
        ignore_class: Optional[int] = None,
        axis: int = -1,
    ):
        super().__init__(
            sparse_categorical_crossentropy,
            name,
            dtype=dtype,
            from_logits=from_logits,
            ignore_class=ignore_class,
            axis=axis,
        )
```

**독립 실행형 사용:**

In [ ]:
# y_true = one_hot(y_true) = [[0, 1, 0], [0, 0, 1]]
# logits = log(y_pred)
# softmax = exp(logits) / sum(exp(logits), axis=-1)
# softmax = [[0.05, 0.95, EPSILON], [0.1, 0.8, 0.1]]
# xent = -sum(y * log(softmax), 1)
# log(softmax) = [[-2.9957, -0.0513, -16.1181],
#                 [-2.3026, -0.2231, -2.3026]]
# y_true * log(softmax) = [[0, -0.0513, 0], [0, 0, -2.3026]]
# xent = [0.0513, 2.3026]
# Reduced xent = (0.0513 + 2.3026) / 2
m = tf.keras.metrics.SparseCategoricalCrossentropy()
m.update_state([1, 2],
               [[0.05, 0.95, 0], [0.1, 0.8, 0.1]])
m.result().numpy()

In [ ]:
m.reset_state()
m.update_state([1, 2],
               [[0.05, 0.95, 0], [0.1, 0.8, 0.1]],
               sample_weight=tf.constant([0.3, 0.7]))
m.result().numpy()

In [ ]:
import tensorflow as tf
tf.keras.losses.SparseCategoricalCrossentropy??

### 5.3 `tf.keras.losses.SparseCategoricalCrossentropy`

- 두 개 이상의 레이블 클래스가 있는 경우 이 교차 엔트로피 측정항목을 사용하라.
  레이블은 정수로 제공될 것으로 예상된다. `원-핫` 표현을 사용하여 레이블을 제공하려면 `CategoricalCrossentropy` 측정항목을 사용하라.
  `y_pred`에 대해 특성당 `# 클래스` 부동소수점 값이 있어야 하고 `y_true`에 대해 특성당 단일 부동소수점 값이 있어야 한다.

- 아래 단편적인 내용에서, `y_true`에 대한 예제당 단일 부동 소수점 값과 `y_pred`에 대한 예제당 `# 클래스` 부동 소수점 값이 있다.
  `y_true`의 형태는 `[batch_size]` 이고 `y_pred`의 형태는 `[batch_size, num_classes]`.


- `compile()` API에서:

```python
model.compile(optimizer='sgd',
              loss=tf.keras.losses.SparseCategoricalCrossentropy())
```

**서명:**

```python
tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=False,
    ignore_class=None,
    reduction='auto',
    name='sparse_categorical_crossentropy',
)
```

- 인자:
  - `from_logits`: `y_pred`가 로짓 텐서로 예상되는지 여부. 기본적으로, `y_pred`가 확률 분포를 인코드한다고 가정.
  - `ignore_class`: 선택적으로 정수. 손실 계산중에 ID는 무시된다.
     이것은 유용한데, 예를 들어, 분할 맵에서 "void" 클래스(일반적으로 -1 또는 255)를 특성으로 하는 분할 문제에 유용하다.
     기본적으로 (`ignore_class=None`), 모든 클래스는 고려된다.
  - `reduction`: 손실에 적용할 `tf.keras.losses.Reduction`의 유형. 기본값은 `AUTO`.
    `AUTO` indicates that the reduction option will be determined by the usage context. 
    대부분이 경우 이것의 기본값은 `SUM_OVER_BATCH_SIZE`이다.
    `tf.distribute.Strategy`이 사용되면, `tf.keras` `compile` 및 `fit`와 같은 내장 훈련 루프 밖에서 `AUTO` 및 `SUM_OVER_BATCH_SIZE` 사용하면 오류가 발생될 것이다. 자세한 사항은 사용자 훈련 [자습서](https://www.tensorflow.org/tutorials/distribute/custom_training) 참조하라.
  - `name`: 인스턴스에 대한 선택적 이름. 기본값은 'sparse_categorical_crossentropy'.

**소스:**

```python
@keras_export("keras.losses.SparseCategoricalCrossentropy")
class SparseCategoricalCrossentropy(LossFunctionWrapper):
    def __init__(
        self,
        from_logits=False,
        ignore_class=None,
        reduction=losses_utils.ReductionV2.AUTO,
        name="sparse_categorical_crossentropy",
    ):
        super().__init__(
            sparse_categorical_crossentropy,
            name=name,
            reduction=reduction,
            from_logits=from_logits,
            ignore_class=ignore_class,
        )
```

**독립 실행형 사용:**

In [ ]:
y_true = [1, 2]
y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
# Using 'auto'/'sum_over_batch_size' reduction type.
scce = tf.keras.losses.SparseCategoricalCrossentropy()
scce(y_true, y_pred).numpy()

In [ ]:
# Calling with 'sample_weight'.
scce(y_true, y_pred, sample_weight=tf.constant([0.3, 0.7])).numpy()

In [ ]:
# Using 'sum' reduction type.
scce = tf.keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.SUM)
scce(y_true, y_pred).numpy()

In [ ]:
# Using 'none' reduction type.
scce = tf.keras.losses.SparseCategoricalCrossentropy(
    reduction=tf.keras.losses.Reduction.NONE)
scce(y_true, y_pred).numpy()

## 6 이진 교차 엔트로피 (backend)
- 출력 텐서와 목표 텐서 사이의 이진 교차 엔트로피

**서명:**
    
```python
tf.keras.backend.binary_crossentropy(target, output, from_logits=False)
```

- 인자:
    - `target`: `output`과 동일 형태를 가지는 텐서.
    - `output`: 텐서.
    - `from_logits`: `output`이 로짓 텐서인지 여부. 기본적으로, `output`이 확률 분포를 인코딩한다고 가정.

- 반환:
    - 텐서.

**소스:**

```python
@keras_export("keras.backend.binary_crossentropy")
@tf.__internal__.dispatch.add_dispatch_support
def binary_crossentropy(target, output, from_logits=False):
    target = tf.convert_to_tensor(target)
    output = tf.convert_to_tensor(output)

    output, from_logits = _get_logits(
        output, from_logits, "Sigmoid", "binary_crossentropy"
    )
    if from_logits:
        return tf.nn.sigmoid_cross_entropy_with_logits(
            labels=target, logits=output
        )

    epsilon_ = _constant_to_tensor(epsilon(), output.dtype.base_dtype)
    output = tf.clip_by_value(output, epsilon_, 1.0 - epsilon_)

    # 확률로 부터 교차 엔트로피 계산.
    bce = target * tf.math.log(output + epsilon())
    bce += (1 - target) * tf.math.log(1 - output + epsilon())
    return -bce
```

## 7 이진 교차 엔트로피 (metrics & losses)
- 이진 교차 엔트로피 손실 계산

### 7.1 `tf.keras.[metrics|losses].binary_crossentropy`

**서명:**

```python
binary_crossentropy(
    y_true,
    y_pred,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
)
```

- 인자:
  - `y_true`: 실측값. 형태 = `[batch_size, d0, .. dN]`.
  - `y_pred`: 예측값. 형태 = `[batch_size, d0, .. dN]`.
  - `from_logits`: `y_pred`가 로짓 텐서인지 여부. 기본적으로, `y_pred`가 확률 분포를 인코딩한다고 가정.
  - `label_smoothing`: [0, 1] 범위의 부동소수점. `0`보다 크면 레이블을 0.5 방향으로 압축하여 레이블을 부드럽게 한다.
    즉, 대상 클래스에 대하여 `1. - 0.5 * label_smoothing`, 비대상 클래스에 대하여 `0.5 * label_smoothing`.
  - `axis`: 평균이 계산되는 축. 기본값은 -1.

- 반환:
  - 이진 교차 엔트로피 손실 값. 형태 = `[batch_size, d0, .. dN-1]`.


**소스:**

```python

@keras_export(
    "keras.metrics.binary_crossentropy", "keras.losses.binary_crossentropy"
)
@tf.__internal__.dispatch.add_dispatch_support
def binary_crossentropy(
    y_true, y_pred, from_logits=False, label_smoothing=0.0, axis=-1
):

    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    label_smoothing = tf.convert_to_tensor(label_smoothing, dtype=y_pred.dtype)

    def _smooth_labels():
        return y_true * (1.0 - label_smoothing) + 0.5 * label_smoothing

    y_true = tf.__internal__.smart_cond.smart_cond(
        label_smoothing, _smooth_labels, lambda: y_true
    )

    return backend.mean(
        backend.binary_crossentropy(y_true, y_pred, from_logits=from_logits),
        axis=axis,
    )
```

**표준 사용법:**

In [ ]:
y_true = [[0, 1], [0, 0]]
y_pred = [[0.6, 0.4], [0.4, 0.6]]
loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
assert loss.shape == (2,)
loss.numpy()

### 7.2 `tf.keras.metrics.BinaryCrossentropy`
- 레이블과 예측 사이의 교차 엔트로피 행렬을 계산
- 이것은 오직 (0과 1) 두 레이블 클래스만 있는 경우에 사용되는 교차 엔트로피 측정항목 클래스이다.

**`compile()` API 에서 사용:**

```python
model.compile(
    optimizer='sgd',
    loss='mse',
    metrics=[tf.keras.metrics.BinaryCrossentropy()])
```

**서명:**

```python
tf.keras.metrics.BinaryCrossentropy(
    name='binary_crossentropy',
    dtype=None,
    from_logits=False,
    label_smoothing=0,
)
```

- 인자:
  - `name`: (선택사양) 측정항목 인스턴스의 문자열 이름.
  - `dtype`: (선택사양) 측정항목 결과의 데이터 유형.
  - `from_logits`: (선택사양) 출력이 로직 텐서 여부.
    기본적으로, 출력은 확률 분포를 인코딩한다고 간주.
  - `label_smoothing`: (선택사양) 범위의 부동소수점.
    0 보다 크면, 레이블 값은 부드러워지고, 레이블 값에 대한 신뢰도가 완화됨을 의미한다.
    예를 들면, `label_smoothing=0.2`은 레이블 `0`에 대하여 `0.1`을, 레이블 `1`에 대하여 `0.9`을 사용할 것이다.

**소스:**

```python
@keras_export("keras.metrics.BinaryCrossentropy")
class BinaryCrossentropy(base_metric.MeanMetricWrapper):
    @dtensor_utils.inject_mesh
    def __init__(
        self,
        name="binary_crossentropy",
        dtype=None,
        from_logits=False,
        label_smoothing=0,
    ):
        super().__init__(
            binary_crossentropy,
            name,
            dtype=dtype,
            from_logits=from_logits,
            label_smoothing=label_smoothing,
        )

```

**독립 실행형 사용:**

In [ ]:
m = tf.keras.metrics.BinaryCrossentropy()
m.update_state([[0, 1], [0, 0]], [[0.6, 0.4], [0.4, 0.6]])
m.result().numpy()

In [ ]:
m.reset_state()
m.update_state([[0, 1], [0, 0]], [[0.6, 0.4], [0.4, 0.6]], sample_weight=[1, 0])
m.result().numpy()

### 7.3 `tf.keras.losses.BinaryCrossentropy`
- 레이블과 예측 사이의 교차 엔트로피 행렬을 계산
- 이진 (0 또는 1) 분류 애플리케이션에 대하여 이 교차-엔트로피 손실을 사용하라.
- 손실 함수는 다음 입력을 요구한다:
    - `y_true` (실측 값): 이것은 0 또는 1.
    - `y_pred` (예측 값): 이것은 모델의 예측, 가령, [로짓](https://en.wikipedia.org/wiki/Logit) 또는 확률을 나타내는 단일 부동소수점 값.
       (`from_logits=True`이면 [-inf, inf] 범위의 값, `from_logits=False`이면 [0., 1.]의 값)
       
      **권장 사용법:** (`from_logits=True` 설정)

**`tf.keras` API 에서:**

```python
model.compile(
  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
  ....
)
```

**서명:**

```python
tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction='auto',
    name='binary_crossentropy',
)
```

- 인자:
  - `from_logits`: `y_pred`를 [로짓](https://en.wikipedia.org/wiki/Logit) 값으로 해석할지 여부.
    기본적으로, `y_pred`이 확률을 담고 있다고 가정 (가령, [0, 1] 범위 값). 
  - `label_smoothing`: [0, 1] 범위의 부동소수점. 0 이면, 스무딩이 발생하지 않는다. 
    0 보다 크면, 예측 레이블과 실제 레이블의 스무딩 버전 사이의 손실을 계산한다. 여기서 스무딩은 레이블을 0.5로 압축한다.
    `label_smoothing`의 값이 클수록 스무딩이 심해진다.
  - `axis`: 교차 엔트로피를 계산할 축(특성 축). 기본값은 -1.
  - `reduction`: 손실에 적용할 `tf.keras.losses.Reduction`의 유형. 기본값은 `AUTO`.
     `AUTO`는 축소 옵션이 사용 컨텍스트에 따라 결정됨을 나타낸다. 대부분의 경우, 기본값은 `SUM_OVER_BATCH_SIZE`이다. 
     `tf.distribute.Strategy`와 함께 사용하는 경우, `tf.keras` `compile` 및 `fit`과 같은 내장 훈련 루프 외부에서 `AUTO` 또는 `SUM_OVER_BATCH_SIZE`를 사용하면 오류가 발생한다.
     상세한 사항은 이 사용자 훈련 [자습서](https://www.tensorflow.org/tutorials/distribute/custom_training) 참조하라.
  - `name`: 연산에 대한 이름. 기본 값은 'binary_crossentropy'.

**소스:**

```python
@keras_export("keras.losses.BinaryCrossentropy")
class BinaryCrossentropy(LossFunctionWrapper):
    def __init__(
        self,
        from_logits=False,
        label_smoothing=0.0,
        axis=-1,
        reduction=losses_utils.ReductionV2.AUTO,
        name="binary_crossentropy",
    ):
        super().__init__(
            binary_crossentropy,
            name=name,
            reduction=reduction,
            from_logits=from_logits,
            label_smoothing=label_smoothing,
            axis=axis,
        )
        self.from_logits = from_logits
```

**독립 실행형 사용:**

In [ ]:
# Example 1: (batch_size = 1, number of samples = 4)
y_true = [0, 1, 0, 0]
y_pred = [-18.6, 0.51, 2.94, -12.8]
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bce(y_true, y_pred).numpy()

In [ ]:
# Example 2: (batch_size = 2, number of samples = 4)
y_true = [[0, 1], [0, 0]]
y_pred = [[-18.6, 0.51], [2.94, -12.8]]
# Using default 'auto'/'sum_over_batch_size' reduction type.
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True)
bce(y_true, y_pred).numpy()

In [ ]:
# Using 'sample_weight' attribute
bce(y_true, y_pred, sample_weight=[0.8, 0.2]).numpy()

In [ ]:
# Using 'sum' reduction` type.
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM)
bce(y_true, y_pred).numpy()

In [ ]:
# Using 'none' reduction type.
bce = tf.keras.losses.BinaryCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
bce(y_true, y_pred).numpy()

**기본 사용법:** (`from_logits=False` 설정)

In [ ]:
# Make the following updates to the above "Recommended Usage" section
# 1. Set `from_logits=False`
tf.keras.losses.BinaryCrossentropy() # OR ...('from_logits=False')
# 2. Update `y_pred` to use probabilities instead of logits
y_pred = [0.6, 0.3, 0.2, 0.8] # OR [[0.6, 0.3], [0.2, 0.8]]

## 8 평균 제곱 오차
- Mean Squared Error, MSE

### 7.1 `tf.keras.[metrics|losses].mse`
- 레이블과 예측 사이의 평균 제곱 오차를 계산
- 입력 사이의 제곱 거리를 계산한 후, 마지막 차원의 평균 값이 반환
- `loss = mean(square(y_true - y_pred), axis=-1)`

**서명:**

```python
tf.keras.metrics.mse(y_true, y_pred)
```

- 인자:
  - `y_true`: 실측값. 형태 = `[batch_size, d0, .. dN]`.
  - `y_pred`: 예측값. 형태 = `[batch_size, d0, .. dN]`.

- 반환:
  - 평균 제곱 오차 값. 형태 = `[batch_size, d0, .. dN-1]`.


**소스:*

```python
@keras_export(
    "keras.metrics.mean_squared_error",
    "keras.metrics.mse",
    "keras.metrics.MSE",
    "keras.losses.mean_squared_error",
    "keras.losses.mse",
    "keras.losses.MSE",
)
@tf.__internal__.dispatch.add_dispatch_support
def mean_squared_error(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    return backend.mean(tf.math.squared_difference(y_pred, y_true), axis=-1)
```

**독립 실행형 사용**

In [ ]:
y_true = np.random.randint(0, 2, size=(2, 3))
y_pred = np.random.random(size=(2, 3))
loss = tf.keras.losses.mean_squared_error(y_true, y_pred)
assert loss.shape == (2,)
assert np.array_equal(loss.numpy(), np.mean(np.square(y_true - y_pred), axis=-1))

## 9 평균 절대 오차
- Mean Absolute Error, MAE
- 레이블과 예측 사이의 평균 절대 오차를 계산
- `loss = mean(abs(y_true - y_pred), axis=-1)`

### 8.1 `tf.keras.[metrics|losses].mae`

**서명:**

```python
tf.keras.losses.mae(y_true, y_pred)
```

- 인자:
  - `y_true`: 실측값. 형태 = `[batch_size, d0, .. dN]`.
  - `y_pred`: 예측값. 형태 = `[batch_size, d0, .. dN]`.

- 반환:
  - 평균 제곱 오차 값. 형태 = `[batch_size, d0, .. dN-1]`.

**소스:**

```python
@keras_export(
    "keras.metrics.mean_absolute_error",
    "keras.metrics.mae",
    "keras.metrics.MAE",
    "keras.losses.mean_absolute_error",
    "keras.losses.mae",
    "keras.losses.MAE",
)
@tf.__internal__.dispatch.add_dispatch_support
def mean_absolute_error(y_true, y_pred):
    y_pred = tf.convert_to_tensor(y_pred)
    y_true = tf.cast(y_true, y_pred.dtype)
    return backend.mean(tf.abs(y_pred - y_true), axis=-1)
```

**독립 실행형 사용:**

In [ ]:
y_true = np.random.randint(0, 2, size=(2, 3))
y_pred = np.random.random(size=(2, 3))
loss = tf.keras.losses.mean_absolute_error(y_true, y_pred)
assert loss.shape == (2,)
assert np.array_equal(loss.numpy(), np.mean(np.abs(y_true - y_pred), axis=-1))

## 10 평균 제곱근 전파
- Root Mean Square Propagation, RMSprop
- RMSprop 요점은 다음과 같다:
  - 경사 제곱의 이동 평균 유지한다.
  - 경사를 이 평균의 근으로 나눈다.
- 참고: [Hinton, 2012](http://www.cs.toronto.edu/~tijmen/csc321/slides/lecture_slides_lec6.pdf)

### 10.1 `tf.keras.optimizers.RMSprop`

**서명:**

```python
tf.keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.0,
    epsilon=1e-07,
    centered=False,
    name='RMSprop',
    **kwargs,
)
```

- 인자:
  - `learning_rate`: `Tensor`, 부동소수점 값, 또는 
    `tf.keras.optimizers.schedules.LearningRateSchedule`로 예정된 것, 
    또는 인자를 받지 않고 사용할 실제 값을 반환하는 호출가능. 학습 비율. 기본값은 0,001.
  - `rho`: 히스토리/도래되는 경사에 대한 할인 요소. 기본값은 0.9.
  - `momentum`: 스칼라 또는 스칼라 `Tensor`. 기본값은 0.0.
  - `epsilon`: 수치 안정성을 위한 작은 상수. 
    이 엡실론은 알고리즘 1의 엡실론이 아니라 Kingma 과 Ba 논문(섹션 2.1 직전 공식)의 "엡실론 햇"이다. 기본값은 1e-7.
  - `centered`: 참/거짓. If `True`, 경사는 경사의 예측된 변수에 의해 정규화된다; 거짓이면, 집중되지 않은 두 번째 순간에 의해.
    이것을 'True'로 설정하면 학습에 도움이 될 수 있지만 계산 및 메모리 측면에서 약간 더 비싸다. 기본값은 `False`.
  - `name`: 경사를 적용할 때 생성되는 연사에 대한 선택적 이름 접두사. 기본값은 `"RMSprop"`.
  - `**kwargs`: 키워드 인자. keyword arguments. 허용되는 인자는 `clipvalue`, `clipnorm`, `global_clipnorm` 이다.
    `clipvalue` (부동소수점) 설정되면, 각 가중치의 경사는 이 값보다 높지 않도록 잘린다.
    `clipnorm` (부동소수점) 설정되면, 각 가중치의 경사는 개별적으로 잘려서 정규값이이 이 값보다 높지 않다.
    `global_clipnorm` (부동소수점) 설정되면, 전역 정규값이 이 값보다 높지 않도록 모든 가중치의 경사가 잘린다.


- 이 RMSprop 구현은 Nesterov 모멘텀이 아닌 일반 모멘텀을 사용한다. 

**사용법:**

In [ ]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)
var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2) / 2.0    # d(loss) / d(var1) = var1
step_count = opt.minimize(loss, [var1]).numpy()
var1.numpy()

## 11 정확도 (측정항목)

### 11.1 `tf.keras.metrics.Accuracy`

- 예측이 레이블과 동일한 빈도를 계산

- 이 측정항목은 `y_pred`가 `y_true`와 일치하는 빈도를 계산하는 데 사용되는 `total` 및 `count`라는 두 개의 로컬 변수를 생성한다.
  이 빈도는 궁극적으로 `binary accuracy` 반환한다: 단순히 `total`을 `count`로 나누는 멱등성 연산이다.

- `sample_weight`가 `None`이면, 가중치 기본값은 1이다. 0의 `sample_weight`를 사용하여 값을 마스크한다.



- Usage with `compile()` API:

```python
model.compile(optimizer='sgd',
              loss='mse',
              metrics=[tf.keras.metrics.Accuracy()])
```

**서명:**

```python
tf.keras.metrics.Accuracy(name='accuracy', dtype=None)
```

- 인자:
  - `name`: (선택적) 측정 항목 인스턴스의 문자열 이름.
  - `dtype`: (선택적) 측정 항목의 데이터 유형.

**소스:**

```python
@keras_export("keras.metrics.Accuracy")
class Accuracy(base_metric.MeanMetricWrapper):
    @dtensor_utils.inject_mesh
    def __init__(self, name="accuracy", dtype=None):
        super().__init__(accuracy, name, dtype=dtype)
```

**독립 실행형 사용:**

In [ ]:
m = tf.keras.metrics.Accuracy()
m.update_state([[1], [2], [3], [4]], [[0], [2], [3], [4]])
m.result().numpy()

In [ ]:
m.reset_state()
m.update_state([[1], [2], [3], [4]], [[0], [2], [3], [4]], sample_weight=[1, 1, 0, 0])
m.result().numpy()

### 11.2 ` tf.keras.metrics.BinaryAccuracy`

- 이진 레이블에 일치하는 예측 빈도 계산.

- 이 측정항목은 `y_pred`가 `y_true`와 일치하는 빈도를 계산하는 데 사용되는 `total` 및 `count`라는 두 개의 로컬 변수를 생성한다.
  이 빈도는 궁극적으로 `binary accuracy` 반환한다: 단순히 `total`을 `count`로 나누는 멱등성 연산이다.

- `sample_weight`가 `None`이면, 가중치 기본값은 1이다. 0의 `sample_weight`를 사용하여 값을 마스크한다.

    
- `compile()` API 사용례:

```python
model.compile(optimizer='sgd',
              loss='mse',
              metrics=[tf.keras.metrics.BinaryAccuracy()])
```

**서명:**

```python
tf.keras.metrics.BinaryAccuracy(
    name='binary_accuracy',
    dtype=None,
    threshold=0.5,
)
```

- 인자:
  - `name`: (선택적) 측정 항목 인스턴스의 문자열 이름.
  - `dtype`: (선택적) 측정 항목 결과의 데이터 유형.
  - `threshold`: (선택적) 예측 값이 1 또는 0인지 결정하기 위한 임계값을 나타내는 부동소수점.

**소스:**

```python
@keras_export("keras.metrics.BinaryAccuracy")
class BinaryAccuracy(base_metric.MeanMetricWrapper):
    @dtensor_utils.inject_mesh
    def __init__(self, name="binary_accuracy", dtype=None, threshold=0.5):
        super().__init__(
            metrics_utils.binary_matches, name, dtype=dtype, threshold=threshold
        )
```

**독립 실행형 사용:**

In [ ]:
m = tf.keras.metrics.BinaryAccuracy()
m.update_state([[1], [1], [0], [0]], [[0.98], [1], [0], [0.6]])
m.result().numpy()

In [ ]:
m.reset_state()
m.update_state([[1], [1], [0], [0]], [[0.98], [1], [0], [0.6]], sample_weight=[1, 0, 0, 1])
m.result().numpy()